## データ準備

In [ ]:
import urllib.request
import sys

#ダウンロード
url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
urllib.request.urlretrieve(url,"ldcc-20140209.tar.gz")
# 解凍
!tar xvf ldcc-20140209.tar.gz

In [ ]:
import pandas as pd
import os
from glob import glob
import linecache

# カテゴリを配列で取得
categories = [name for name in os.listdir('text') if os.path.isdir(f'text/{name}')]
print('カテゴリ：', categories)

title_list = []
body_list = []
cat_list = []
for category in categories:
  path = f'text/{category}/*.txt'
  files = glob(path)
  for text_name in files:
    # 各テキストの3行目をリストに追加（タイトル）
    title_list.append(linecache.getline(text_name, 3))
    # 各テキストの4〜10行目をリストに追加（本文）
    body = ''
    for body_line in range(4, 10):
      body += linecache.getline(text_name, body_line)
    body_list.append(body)
    # 各テキストのカテゴリーをリストに追加
    cat_list.append(category)

cols = {'TITLE': title_list, 'BODY': body_list, 'CATEGORY': cat_list}
df = pd.DataFrame(cols)
df.head()

## データを眺める

In [ ]:
len(df)

In [ ]:
df['CATEGORY'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df['CATEGORY'].value_counts().plot(kind='barh')

In [ ]:
df['CATEGORY'].value_counts().plot.kde()

## 前処理

In [ ]:
import re

def preprocessing(text):
  # 全角 => 半角
  text = text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
  # 英語大文字を小文字化
  text = text.lower()
  # 削除する文字列
  remove_list = ['\n', '　', ' ']
  for rem_word in remove_list:
    text = text.replace(rem_word, '')
  # 数字列を0に置換
  text = re.sub('[0-9]+', '0', text)

  return text

In [ ]:
df['TITLE'] = df['TITLE'].map(preprocessing)

In [ ]:
df.head()

## Mecabインストール

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

!sed -e "s!/var/lib/mecab/dic/debian!/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd!g" /etc/mecabrc &gt; /etc/mecabrc.new
!cp /etc/mecabrc /etc/mecabrc.org
!cp /etc/mecabrc.new /etc/mecabrc

## 分かち書き

In [ ]:
import MeCab

def tokenize(text, target_pos=['名詞', '形容詞', '形容動詞', '動詞', '副詞']):
  tokens = []
  mecab = MeCab.Tagger ('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
  mecab.parse('') #文字列がGCされるのを防ぐ
  node = mecab.parseToNode(text)
  while node:
    #単語を取得
    word = node.surface
    #品詞を取得
    pos = node.feature.split(",")[0]
    # 名詞の場合のみ抽出
    if pos in target_pos:
      tokens.append(word)
    #次の単語に進める
    node = node.next
  return tokens

In [ ]:
df['TITLE_WAKATI'] = df['TITLE'].map(lambda x: ' '.join(tokenize(x)))
df.head()

## 目的変数をIDに変換

In [ ]:
ctgs_dict = {}
for idx, ctg in enumerate(categories):
  ctgs_dict[ctg] = idx

df['CATEGORY_CODE'] = df['CATEGORY'].map(ctgs_dict)
df[::777]

## 単語埋め込みのために、学習・検証・テストに分割

In [ ]:
from sklearn.model_selection import train_test_split

train, valid_test = train_test_split(df, test_size=0.2, random_state=144, stratify=df['CATEGORY_CODE'])
valid, test = train_test_split(valid_test, test_size=0.5, random_state=144, stratify=valid_test['CATEGORY_CODE'])

# 事例数の確認
print('===学習データ===')
print(train['CATEGORY_CODE'].value_counts())
print('===検証データ===')
print(valid['CATEGORY_CODE'].value_counts())
print('===評価データ===')
print(test['CATEGORY_CODE'].value_counts())

## Word2Vecによる単語埋め込み

In [ ]:
wakati = train[['TITLE_WAKATI']]
wakati.to_csv('wakati.csv', sep='\n', header=True, index=False)
wakati

In [ ]:
from gensim.models import word2vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentence_data = word2vec.LineSentence('wakati.csv')
vectorizer = word2vec.Word2Vec(sentence_data,
                         sg=1,        # Skip-gram
                         size=200,    # 次元数
                         min_count=1, # min_count回未満の単語を破棄
                         window=3,    # 文脈の最大単語数
                         hs=1,        # 階層ソフトマックス(ネガティブサンプリングするなら0)
                         negative=5,  # ネガティブサンプリング
                         iter=50      # Epoch数
                         )

In [ ]:
vectorizer.most_similar('映画')

## 単語埋め込みから特徴ベクトル作成

In [ ]:
import string
import torch

def transform_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()  # 記号をスペースに置換後、スペースで分割してリスト化
  vec = [vectorizer[word] for word in words if word in vectorizer]  # 1語ずつベクトル化

  return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

In [ ]:
# 特徴ベクトルの作成
X_train = torch.stack([transform_w2v(text) for text in train['TITLE_WAKATI']])
X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE_WAKATI']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE_WAKATI']])

print(X_train.size())
print(X_train)

## ラベルベクトルの作成

In [ ]:
y_train = torch.tensor(train['CATEGORY_CODE'].values)
y_valid = torch.tensor(valid['CATEGORY_CODE'].values)
y_test = torch.tensor(test['CATEGORY_CODE'].values)

print(y_train.size())
print(y_train)

## 学習 => 予測 => 精度評価

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def learn_pred_accuracy(clf, train_x, train_y, valid_x, valid_y, test_x, test_y):
  # 学習
  clf.fit(train_x, train_y)
  # 予測
  predict_vy = clf.predict(valid_x)
  predict_y = clf.predict(test_x)
  # 正解率
  print('検証True:', np.count_nonzero(predict_vy == valid_y))
  print('検証False:', np.count_nonzero(predict_vy != valid_y))
  print('検証正解率:', np.count_nonzero(predict_vy == valid_y) / len(valid_y))
  print('テストTrue:', np.count_nonzero(predict_y == test_y))
  print('テストFalse:', np.count_nonzero(predict_y != test_y))
  print('テスト正解率:', np.count_nonzero(predict_y == test_y) / len(test_y))
  # 様々な精度結果
  print(classification_report(test_y, predict_y))
  # 正解と予測データ
  print('検証正解：', valid_y[:20])
  print('検証予測：', predict_vy[:20])
  print('test正解：', test_y[:20])
  print('test予測：', predict_y[:20])
  # 予測データのヒストグラム
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
  sns.countplot(x=predict_vy, ax=ax1)
  sns.countplot(x=predict_y, ax=ax2)
  # 混合行列
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 5))
  cm = confusion_matrix(valid_y, predict_vy)
  sns.heatmap(cm, cmap='jet', annot=True, ax=ax1)
  cm = confusion_matrix(test_y, predict_y)
  sns.heatmap(cm, cmap='jet', annot=True, ax=ax2)

## Naive Bayes

In [ ]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn import preprocessing

clf = MultinomialNB(alpha=.01)
mm = preprocessing.MinMaxScaler()
X_train_mm = mm.fit_transform(X_train)
X_valid_mm = mm.transform(X_valid)
X_test_mm = mm.transform(X_test)
mm = preprocessing.MinMaxScaler()
learn_pred_accuracy(clf, X_train_mm, y_train, X_valid_mm, np.array(y_valid), X_test_mm, np.array(y_test))

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=144)
learn_pred_accuracy(clf, X_train, y_train, X_valid, np.array(y_valid), X_test, np.array(y_test))

## XGBoost

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(random_state=144, objective='binary:logistic')
learn_pred_accuracy(clf, X_train, y_train, X_valid, np.array(y_valid), X_test, np.array(y_test))

## ロジスティック回帰

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=144, max_iter=144000)
learn_pred_accuracy(clf, X_train, y_train, X_valid, np.array(y_valid), X_test, np.array(y_test))

## LSTM

In [ ]:
# バッチサイズを1とする
"""
X_train = X_train.view(len(X_train), 1, -1)
X_valid = X_valid.view(len(X_valid), 1, -1)
X_test = X_test.view(len(X_test), 1, -1)

print('X_train:', X_train.size())
print('X_valid:', X_valid.size())
print('X_test:', X_test.size())
"""

In [ ]:
import torch
import torch.nn as nn

# nn.Moduleを継承して新しいクラスを作る。決まり文句
class LSTMClassifier(nn.Module):
  # モデルで使う各ネットワークをコンストラクタで定義
  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    # 親クラスのコンストラクタ。決まり文句
    super(LSTMClassifier, self).__init__()
    # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。
    self.hidden_dim = hidden_dim
    # インプットの単語をベクトル化するために使う
    self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
    # LSTMの隠れ層。これ１つでOK。超便利。
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
    self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
    # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
    self.softmax = nn.LogSoftmax(dim=1)

  # 順伝播処理はforward関数に記載
  def forward(self, sentence):
    # 文章内の各単語をベクトル化して出力。2次元のテンソル
    embeds = self.word_embeddings(sentence)
    # 2次元テンソルをLSTMに食わせられる様にviewで３次元テンソルにした上でLSTMへ流す。
    # 上記で説明した様にmany to oneのタスクを解きたいので、第二戻り値だけ使う。
    _, lstm_out = self.lstm(embeds.view(len(sentence), 1, -1))
    # lstm_out[0]は３次元テンソルになってしまっているので2次元に調整して全結合。
    tag_space = self.hidden2tag(lstm_out[0].view(-1, self.hidden_dim))
    # softmaxに食わせて、確率として表現
    tag_scores = self.softmax(tag_space)
    return tag_scores

In [ ]:
import torch.optim as optim

# trainとtestに分割
traindata, testdata = train_test_split(df, train_size=0.8)
# 単語のベクトル次元数
EMBEDDING_DIM = 10
# 隠れ層の次元数
HIDDEN_DIM = 128
# データ全体の単語数
VOCAB_SIZE = len(word2index)
# 分類先のカテゴリの数
TAG_SIZE = len(categories)
# モデル宣言
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE)
# 損失関数はNLLLoss()を使う。LogSoftmaxを使う時はこれを使うらしい。
loss_function = nn.NLLLoss()
# 最適化の手法はSGDで。lossの減りに時間かかるけど、一旦はこれを使う。
optimizer = optim.SGD(model.parameters(), lr=0.01)

def make_wakati(sentence):
  # MeCabで分かち書き
  mecab = MeCab.Tagger ('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
  mecab.parse('') #文字列がGCされるのを防ぐ
  sentence = mecab.parse(sentence)
  # 半角全角英数字除去
  sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
  # 記号もろもろ除去
  sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
  # スペースで区切って形態素の配列へ
  wakati = sentence.split(" ")
  # 空の要素は削除
  wakati = list(filter(("").__ne__, wakati))
  return wakati

# 単語ID辞書を作成する
word2index = {}
for title in df['TITLE']:
  wakati = make_wakati(title)
  for word in wakati:
    if word in word2index: continue
    word2index[word] = len(word2index)

# 文章を単語IDの系列データに変換
# PyTorchのLSTMのインプットになるデータなので、もちろんtensor型で
def sentence2index(sentence):
  wakati = make_wakati(sentence)
  return torch.tensor([word2index[w] for w in wakati], dtype=torch.long)

category2index = {}
for cat in categories:
  if cat in category2index: continue
  category2index[cat] = len(category2index)

def category2tensor(cat):
  return torch.tensor([category2index[cat]], dtype=torch.long)

# 各エポックの合計loss値を格納する
losses = []
# 10ループ回してみる。（バッチ化とかGPU使ってないので結構時間かかる...）
for epoch in range(10):
    all_loss = 0
    for title, cat in zip(traindata['TITLE'], traindata['CATEGORY']):
        # モデルが持ってる勾配の情報をリセット
        model.zero_grad()
        # 文章を単語IDの系列に変換（modelに食わせられる形に変換）
        inputs = sentence2index(title)
        # 順伝播の結果を受け取る
        out = model(inputs)
        # 正解カテゴリをテンソル化
        answer = category2tensor(cat)
        # 正解とのlossを計算
        loss = loss_function(out, answer)
        # 勾配をセット
        loss.backward()
        # 逆伝播でパラメータ更新
        optimizer.step()
        # lossを集計
        all_loss += loss.item()
    losses.append(all_loss)
    print('epoch', epoch, '\t' , 'loss', all_loss)
print('done.')

In [ ]:
plt.plot(losses)

In [ ]:
# テストデータの母数計算
test_num = len(testdata)
# 正解の件数
a = 0
# 勾配自動計算OFF
with torch.no_grad():
    for title, category in zip(testdata['TITLE'], testdata['CATEGORY']):
        # テストデータの予測
        inputs = sentence2index(title)
        out = model(inputs)

        # outの一番大きい要素を予測結果をする
        _, predict = torch.max(out, 1)

        answer = category2tensor(category)
        if predict == answer:
            a += 1
print("predict : ", a / test_num)